In [58]:
import os
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from evaluation_functions import evaluate
from sentence_transformers import SentenceTransformer

from collections import defaultdict

## Set Up

### Functions

In [38]:
def perform_extraction(model, prompt, text, temperature, max_length):
    # ssh -N -f -L localhost:5000:localhost:5000 username@10.19.2.120
    llamasgard_endpoint = "http://localhost:5000/predict"

    # model = codellama/CodeLlama-34b-Instruct-hf
    # Define the payload
    payload = {
        "input": prompt.format(text),
        "model_id": model,
        "parameters": {
            "temperature": temperature,
            "max_length": max_length
        }
    }
    
    #print(prompt.format(text))

    response = requests.post(llamasgard_endpoint, json=payload)

    # Check the response status code
    if response.status_code != 200:
        print("Error:", response.json())

    #print("\n\n" + str(response.json()) + "\n")
    return response.json().get('response')

def perform_cleanup(extraction, openai_api):
    client = OpenAI(api_key=openai_api)
    
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": ""},
            {
                "role": "user",
                "content": """The following text is an extraction of adverse event terms from a drug label. Please remove any preamble or postamble from the list and turn the list of ADEs into a comma separated list. 
The text: {}""".format(extraction)
            }
        ],
        model="gpt-3.5-turbo-16k",
        temperature=0,
    )
    term = chat_completion.choices[0].message.content
    return term

In [39]:
# row['section_text']
# perform_extraction("google/gemma-7b", system_content, prompt, row['section_text'], 0)


In [40]:
# function for extracting 
def extract_ade_terms(config, model, prompt, text, temperature, max_length):
  extraction = perform_extraction(model, prompt, text, temperature, max_length)
  if extraction is None:
    raise Exception(f"perform_extraction() return None for {model}")
  else:
    extraction = perform_cleanup(extraction, config['OpenAI']['openai_api_key'])
    return extraction


### Variables

In [41]:
drug_file = 'data/TAC2017/train_drug_label_text.csv'
manual_file = 'data/TAC2017/train_drug_label_text_manual_ades.csv'
my_max = 10000

In [42]:
drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[2].split('_')[0] # assuming file follows format "train_..." or "test...."

all_sections = drugs.query("section_name != 'all-concat'").groupby('drug_name')['section_text'].apply(' '.join).reset_index()
all_sections.insert(1, "section_name", ["all-concat" for _ in range(all_sections.shape[0])])
drugs = pd.concat([drugs, all_sections])

## Run Llama

In [43]:
outputs = {}

In [44]:
config = json.load(open('./config.json'))

# gpt_model = 'code-llama-34b'
# model_id = "codellama/CodeLlama-34b-Instruct-hf"
# model_id = "google/gemma-7b"
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

model_name = model_id.split('/')[1]

max_length = 5000

In [45]:
nruns = 1
temperature = 0

system_options = {
    "no-system-prompt": "",
    "pharmexpert-v0": "You are an expert in pharmacology.",
    "pharmexpert-v1": "You are an expert in medical natural language processing, adverse drug reactions, pharmacology, and clinical trials."
}

prompt_options = {
    "fatal-prompt-v2": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
The text is :'{}' 
"""
}

system_name = "pharmexpert-v1"
system_content = system_options[system_name]

user_prompt_name = "fatal-prompt-v2"
user_prompt = prompt_options[user_prompt_name]

gpt_params = [f"temp{temperature}"]

if model_id.split('/')[0] in ("codellama", "mistralai"):
    print("Modifying the prompt to include instruction tags.")
    prefix = ""
    prompt = f"<s>[INST] <<SYS>>\\n{system_content}\\n<</SYS>>\\n\\n{user_prompt}[/INST]{prefix}"
else:
    prompt = system_content + '\n' + user_prompt

output_file_basename = '{}_{}_{}_{}_{}'.format(model_name, user_prompt_name, system_name, '-'.join(gpt_params), set_type)
output_file_basename

Modifying the prompt to include instruction tags.


'Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train'

In [64]:
max_tries = 2
num_tries = defaultdict(int)

In [69]:
# run local model
for i in range(nruns):
    run_key = "{}_run{}".format(output_file_basename, i)
    print(run_key)
    
    if run_key in outputs:
        print(f"Run {run_key} already started will pick up from where it was left off.")
    elif os.path.exists('results/extract/{}.csv'.format(run_key)):
        gpt_output = pd.read_csv('results/extract/{}.csv'.format(run_key))
        outputs[run_key] = gpt_output
        print(f"Run {run_key} started, loading from disk and pick up from where it was left off.")
    
    start = time.time()
    results = list()
    for i, row in tqdm(drugs.iterrows(), total=drugs.shape[0]):

        if num_tries[(run_key,i)] >= max_tries:
            print(f"Skipping run {(run_key,i)} because we have tried it {max_tries} times.")
            continue
        
        name, section = row['drug_name'], row['section_name']

        if run_key in outputs:
            prev_run_results = outputs[run_key].query(f"drug_name == '{name}'").query(f"section_name == '{section}'")
            if prev_run_results.shape[0]==1:
                results.append([name, section, prev_run_results.gpt_output.values[0]])
                continue
        
        text = row['section_text'][:14500]
        try:
            gpt_out = extract_ade_terms(config, model_id, prompt, text, temperature, max_length)
            results.append([name, section, gpt_out])    
        except Exception as err:
            num_tries[(run_key,i)] += 1
            print(f"Encountered an exception for row: {name} {section}.")
            print(f"This is the {num_tries[(run_key,i)]} time we have tried to run this. Will try {max_tries} times and then skip.")
            print(f"Will save progress, so you can restart from where we left off.")
            gpt_output = pd.DataFrame(
                [r for r in results if r is not None],
                columns=['drug_name', 'section_name', 'gpt_output']
            )
            if gpt_output.shape[0] > 0:
                print("Saved progress successfully.")
                outputs[run_key] = gpt_output
                gpt_output.to_csv('results/extract/{}.csv'.format(run_key))
            
            print(f"Failed for prompt: {prompt.format(text)}")
            raise err
            continue
    
    gpt_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    end = time.time()
    
    if gpt_output.shape[0] > 0:
        outputs[run_key] = gpt_output
        gpt_output.to_csv('results/extract/{}.csv'.format(run_key))
    
    print(f"Run: {run_key}, time elapsed: {end-start}s.")

Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0
Run Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0 already started will pick up from where it was left off.


 36%|███▌      | 121/340 [00:00<00:00, 1200.94it/s]

Skipping run ('Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0', 0) because we have tried it 2 times.
Skipping run ('Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0', 192) because we have tried it 2 times.
Skipping run ('Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0', 0) because we have tried it 2 times.


100%|██████████| 340/340 [48:23<00:00,  8.54s/it]  

Run: Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0, time elapsed: 2903.6596310138702s.


## Evaluation

In [70]:
evaluate(outputs, manual_ades, 'strict')
evaluate(outputs, manual_ades, 'lenient')

/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  output['gpt_output'] = output['gpt_output'].str.lower().str.replace('.', '').str.replace('\n-', ', ').str.split(', ')


Running strict evaluation and saving results to disk.
Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0
saving results to results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_strict_granular.csv and results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_strict_overall.csv


100%|██████████| 101/101 [00:01<00:00, 52.37it/s]
/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  output['gpt_output'] = output['gpt_output'].str.lower().str.replace('.', '').str.replace('\n-', ', ').str.split(', ')


Running lenient evaluation and saving results to disk.
Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0
saving results to results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_lenient_granular.csv and results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_lenient_overall.csv


100%|██████████| 101/101 [00:16<00:00,  6.03it/s]


In [72]:
# if using embeddings -- run this once:
# get embeddings for manual annotation --- this part is slow -- but should take <5 min
embed_model_name = 'llmrails/ember-v1'
embed_model = SentenceTransformer(embed_model_name)
man_embeds = embed_model.encode(manual_ades['reaction_string'].tolist())
manual_ades['embeds'] = list(man_embeds)

In [73]:
evaluate(outputs, manual_ades, 'embed', embed_model=embed_model, embed_model_name=embed_model_name)

Running embed evaluation and saving results to disk.
Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0
saving results to results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_ember-v1_granular.csv and results/evals/Mixtral-8x7B-Instruct-v0.1_fatal-prompt-v2_pharmexpert-v1_temp0_train_run0_ember-v1_overall.csv


/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  output['gpt_output'] = output['gpt_output'].str.lower().str.replace('.', '').str.replace('\n-', ', ').str.split(', ')
100%|██████████| 101/101 [01:22<00:00,  1.22it/s]
